In [1]:
# Imports + Configuration
import os
import time
import requests
import pandas as pd
from datetime import datetime, timezone, timedelta

# =========================
# ClickUp Configuration
# =========================
SPACE_ID = "90159483029"
TEAM_ID = "90152198197"  # For template lookups
TARGET_FOLDER_NAME = "50 BLNG | Utility-scale"
TARGET_LIST_NAME = "HTL | Oitilo Mani"
TASK_TEMPLATE_NAME = "HTL | TEMPLATE | Oitilo Mani"
TASK_TEMPLATE_ID = "t-86c7n8hr7"  # Fallback if name lookup fails
CSV_PATH_SIMPLIFIED = "OITILO_ClickUP_WBS_simplified.csv"

CLICKUP_TOKEN = "pk_56660333_WATA0RDNID48ZA30VX30Z2CRSNB16SN3"
DRY_RUN = False
BASE = "https://api.clickup.com/api/v2"

# Authentication
sess = requests.Session()
sess.headers.update({
    "Authorization": CLICKUP_TOKEN,
    "Accept": "application/json",
    "Content-Type": "application/json",
})

In [2]:
# Helper functions (HTTP + time + deps parsing)
def req(method: str, url: str, *, params=None, json=None, ok=(200, 201), retries=6):
    if DRY_RUN:
        print(f"[DRY_RUN] {method} {url} params={params} json={json}")
        return {}
    last = None
    for i in range(retries):
        r = sess.request(method, url, params=params, json=json, timeout=60)
        if r.status_code in ok:
            return r.json() if r.text else {}
        if r.status_code == 429 or r.status_code >= 500:
            wait = min(60, 2 ** i)
            time.sleep(wait)
            last = (r.status_code, r.text[:400])
            continue
        snippet = (r.text or "")[:400]
        raise RuntimeError(f"{method} {url} failed: {r.status_code} {snippet}")
    raise RuntimeError(f"{method} {url} failed after retries. Last error: {last}")

def date_to_ms_midday_utc(date_str: str) -> int:
    d = datetime.strptime(date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc) + timedelta(hours=12)
    return int(d.timestamp() * 1000)

def days_to_ms(days: float) -> int:
    return int(float(days) * 24 * 60 * 60 * 1000)

def split_deps(dep_cell) -> list[str]:
    if dep_cell is None or (isinstance(dep_cell, float) and pd.isna(dep_cell)):
        return []
    s = str(dep_cell).strip()
    if not s or s.lower() == "nan":
        return []
    parts = []
    for chunk in s.replace("；", ";").replace(",", ";").split(";"):
        c = chunk.strip()
        if c:
            parts.append(c)
    return parts

In [3]:
# Main logic: Import tasks from simplified CSV to ClickUp
print("Using token prefix:", (CLICKUP_TOKEN[:6] + "…"))
print("Target: Space ID={}, Folder={}, List={}".format(SPACE_ID, TARGET_FOLDER_NAME, TARGET_LIST_NAME))

# -------- Simplified CSV flow (find folder/list by name, import tasks) --------
# Find folder by name
folders = req("GET", f"{BASE}/space/{SPACE_ID}/folder", params={"archived": "false"}).get("folders", [])
folder_id = None
for f in folders:
    if str(f.get("name", "")).strip().lower() == TARGET_FOLDER_NAME.strip().lower():
        folder_id = str(f.get("id"))
        break
if folder_id is None:
    raise RuntimeError(f"Folder not found: {TARGET_FOLDER_NAME} in space {SPACE_ID}")
print(f"Found Folder: {TARGET_FOLDER_NAME} (id={folder_id})")

# Find list by name
lists = req("GET", f"{BASE}/folder/{folder_id}/list", params={"archived": "false"}).get("lists", [])
list_id = None
for l in lists:
    if str(l.get("name", "")).strip().lower() == TARGET_LIST_NAME.strip().lower():
        list_id = str(l.get("id"))
        break
if list_id is None:
    created = req("POST", f"{BASE}/folder/{folder_id}/list", json={"name": TARGET_LIST_NAME})
    list_id = str(created.get("id"))
print(f"Found List: {TARGET_LIST_NAME} (id={list_id})")

# Find task template by name (using TEAM_ID)
template_id = TASK_TEMPLATE_ID
try:
    templates = req("GET", f"{BASE}/team/{TEAM_ID}/taskTemplate").get("task_templates", [])
    print(f"\nSearching for template: '{TASK_TEMPLATE_NAME}'")
    print(f"Found {len(templates)} templates. Checking...")
    
    for t in templates:
        t_name = str(t.get("name", "")).strip()
        t_id = str(t.get("id", ""))
        print(f"  - '{t_name}' (comparing: '{t_name.lower()}' == '{TASK_TEMPLATE_NAME.strip().lower()}')")
        
        if t_name.lower() == TASK_TEMPLATE_NAME.strip().lower():
            template_id = t_id
            print(f"  ✓ MATCH FOUND!")
            break
    
    if template_id == TASK_TEMPLATE_ID:
        print(f"\n⚠ Template '{TASK_TEMPLATE_NAME}' not found, using fallback: {TASK_TEMPLATE_ID}")
    else:
        print(f"\n✓ Found Template: {TASK_TEMPLATE_NAME} (id={template_id})")
except Exception as e:
    print(f"Could not search templates, using fallback: {TASK_TEMPLATE_ID} ({e})")
print(f"Using Template ID: {template_id}")

# Load CSV
df_s = pd.read_csv(CSV_PATH_SIMPLIFIED)
required = ["task name", "Dependencies", "Estimated Time (days)", "Start Date", "Due Date", "Description"]
missing = [c for c in required if c not in df_s.columns]
if missing:
    raise ValueError(f"CSV missing columns: {missing}. Found: {list(df_s.columns)}")

# Clean data
df_s = df_s.copy()
df_s["task name"] = df_s["task name"].astype(str).str.strip()
df_s["Dependencies"] = df_s["Dependencies"].astype(str)
df_s["Estimated Time (days)"] = df_s["Estimated Time (days)"].apply(lambda x: None if pd.isna(x) else x)
df_s["Start Date"] = df_s["Start Date"].astype(str).str.strip()
df_s["Due Date"] = df_s["Due Date"].astype(str).str.strip()
df_s["Description"] = df_s["Description"].astype(str).fillna("").str.strip()

# Sort by Start Date, then task name
def safe_dt_s(s):
    try:
        return datetime.strptime(s, "%Y-%m-%d")
    except:
        return datetime(2100, 1, 1)

df_s = df_s.sort_values(by=["Start Date", "task name"], key=lambda col: col.map(safe_dt_s) if col.name == "Start Date" else col).reset_index(drop=True)

print(f"Loaded {len(df_s)} tasks from {CSV_PATH_SIMPLIFIED}")

# Create tasks from template and update fields
name_to_id_s = {}
created_cnt_s = 0
updated_cnt_s = 0

for _, r in df_s.iterrows():
    name = r["task name"]
    created = req(
        "POST",
        f"{BASE}/list/{list_id}/taskTemplate/{template_id}",
        json={"name": name},
        ok=(200, 201),
    )
    task_id = str(created.get("id") or created.get("task", {}).get("id") or "")
    if not task_id:
        raise RuntimeError(f"Could not parse created task id for '{name}'. Response: {created}")
    name_to_id_s[name] = task_id
    created_cnt_s += 1

    # Build payload for field updates
    payload = {}
    
    # Start Date
    sd = r["Start Date"]
    if sd and sd.lower() != "nan":
        payload["start_date"] = date_to_ms_midday_utc(sd)
        payload["start_date_time"] = False
    
    # Due Date
    dd = r["Due Date"]
    if dd and dd.lower() != "nan":
        payload["due_date"] = date_to_ms_midday_utc(dd)
        payload["due_date_time"] = False
    
    # Estimated Time (days)
    est = r["Estimated Time (days)"]
    if est is not None and not (isinstance(est, float) and pd.isna(est)):
        try:
            payload["time_estimate"] = days_to_ms(float(est))
        except:
            pass
    
    # Description - append to template description
    desc_extra = r.get("Description", "")
    if desc_extra and desc_extra.lower() != "nan":
        base_desc = created.get("description") or created.get("task", {}).get("description") or ""
        combined_desc = (base_desc + "\n\n" + desc_extra).strip() if base_desc else desc_extra
        payload["description"] = combined_desc
    
    # Update task with all fields
    if payload:
        req("PUT", f"{BASE}/task/{task_id}", json=payload, ok=(200,))
        updated_cnt_s += 1

print(f"Created: {created_cnt_s} tasks")
print(f"Updated: {updated_cnt_s} tasks with field values")

# Link dependencies
deps_added_s = 0
deps_missing_s = 0
deps_already_s = 0

for _, r in df_s.iterrows():
    child_name = r["task name"]
    child_id = name_to_id_s.get(child_name)
    if not child_id:
        continue
    
    for parent_name in split_deps(r["Dependencies"]):
        parent_id = name_to_id_s.get(parent_name)
        if not parent_id:
            deps_missing_s += 1
            continue
        try:
            req(
                "POST",
                f"{BASE}/task/{child_id}/dependency",
                json={"depends_on": parent_id},
                ok=(200, 201),
            )
            deps_added_s += 1
        except RuntimeError as e:
            if "already" in str(e).lower() or "exist" in str(e).lower():
                deps_already_s += 1
            else:
                raise

print(f"Dependencies linked: {deps_added_s}")
if deps_missing_s > 0:
    print(f"  (Missing parent tasks: {deps_missing_s})")
if deps_already_s > 0:
    print(f"  (Already existed: {deps_already_s})")

print("\n✓ Import complete!")

Using token prefix: pk_566…
Target: Space ID=90159483029, Folder=50 BLNG | Utility-scale, List=HTL | Oitilo Mani
Found Folder: 50 BLNG | Utility-scale (id=901513698403)
Found List: HTL | Oitilo Mani (id=901520291227)

Searching for template: 'HTL | TEMPLATE | Oitilo Mani'
Found 0 templates. Checking...

⚠ Template 'HTL | TEMPLATE | Oitilo Mani' not found, using fallback: t-86c7n8hr7
Using Template ID: t-86c7n8hr7


ValueError: CSV missing columns: ['task name']. Found: ['WBS', 'Branch', 'Task Name', 'Dependencies', 'Estimated Time (days)', 'Start Date', 'Due Date', 'Description']

In [ ]:
# List all available templates and find the target one
print("=" * 60)
print("AVAILABLE TASK TEMPLATES")
print("=" * 60)

found_template_id = None

try:
    templates = req("GET", f"{BASE}/team/{TEAM_ID}/taskTemplate").get("task_templates", [])
    if templates:
        for t in templates:
            template_id_temp = t.get("id")
            template_name = t.get("name")
            print(f"  • {template_name:<50} (id: {template_id_temp})")
            
            # Check if this is the template we're looking for
            if str(template_name).strip().lower() == TASK_TEMPLATE_NAME.strip().lower():
                found_template_id = template_id_temp
                print(f"    ✓ MATCH for '{TASK_TEMPLATE_NAME}'")
        
        print(f"\nTotal: {len(templates)} templates")
        
        if found_template_id:
            print(f"\n✓ Found target template: '{TASK_TEMPLATE_NAME}'")
            print(f"  Template ID: {found_template_id}")
        else:
            print(f"\n⚠ Target template '{TASK_TEMPLATE_NAME}' NOT FOUND")
    else:
        print("No templates found")
except Exception as e:
    print(f"Error fetching templates: {e}")

print("=" * 60)

AVAILABLE TASK TEMPLATES
No templates found


In [6]:
# DEBUG: Test req() function directly
print("Testing req() function...")
print(f"BASE: {BASE}")
print(f"TEAM_ID: {TEAM_ID}")
print(f"Session headers: {sess.headers}")

test_url = f"{BASE}/team/{TEAM_ID}/taskTemplate"
print(f"\nCalling: {test_url}")

result = req("GET", test_url)
print(f"\nResult type: {type(result)}")
print(f"Result keys: {result.keys() if isinstance(result, dict) else 'N/A'}")
print(f"Result: {result}")

Testing req() function...
BASE: https://api.clickup.com/api/v2
TEAM_ID: 90152198197
Session headers: {'User-Agent': 'python-requests/2.32.5', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': 'application/json', 'Connection': 'keep-alive', 'Authorization': 'pk_56660333_WATA0RDNID48ZA30VX30Z2CRSNB16SN3', 'Content-Type': 'application/json'}

Calling: https://api.clickup.com/api/v2/team/90152198197/taskTemplate

Result type: <class 'dict'>
Result keys: dict_keys(['templates'])
Result: {'templates': [{'name': 'TEMPLATE | Sheduled Task Parent', 'id': 't-86c7gd00b'}, {'name': 'TEMPLATE | Scheduled Task Child', 'id': 't-86c7gd05q'}, {'name': 'TEMPLATE | Monitoring Parent', 'id': 't-86c7gd0n2'}, {'name': 'TEMPLATE | Reporting Parent', 'id': 't-86c7gd0ta'}, {'name': 'TEMPLATE | Claim', 'id': 't-86c7gd0w3'}, {'name': 'TEMPLATE | General', 'id': 't-86c7gd0y3'}, {'name': 'TEMPLATE | Monitoring Child', 'id': 't-86c7gd0pb'}, {'name': 'TEMPLATE | HQ General Task', 'id': 't-86c7gngg7'}, {'name':

In [15]:
import requests

url = "https://api.clickup.com/api/v2/team/90152198197/taskTemplate"

headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "pk_56660333_WATA0RDNID48ZA30VX30Z2CRSNB16SN3"
}

response = requests.get(url, headers=headers)

print(response.text)

{"templates":[{"name":"TEMPLATE | Sheduled Task Parent","id":"t-86c7gd00b"},{"name":"TEMPLATE | Scheduled Task Child","id":"t-86c7gd05q"},{"name":"TEMPLATE | Monitoring Parent","id":"t-86c7gd0n2"},{"name":"TEMPLATE | Reporting Parent","id":"t-86c7gd0ta"},{"name":"TEMPLATE | Claim","id":"t-86c7gd0w3"},{"name":"TEMPLATE | General","id":"t-86c7gd0y3"},{"name":"TEMPLATE | Monitoring Child","id":"t-86c7gd0pb"},{"name":"TEMPLATE | HQ General Task","id":"t-86c7gngg7"},{"name":"DEV | Typical Task","id":"t-86c7n8hr7"},{"name":"Project Intake (Dev Portfolio GR)","id":"t-86c7mz2n1"},{"name":"TEMPLATE | Corrective","id":"t-86c7gcxtr"},{"name":"HTL | TEMPLATE | Oitilo Mani","id":"t-86c7t51ff"}]}
